## read url from csv

In [124]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [36]:
url = pd.read_csv('output/url.csv')

## scraping address from footer 

In [86]:
import urllib
import pandas as pd
from bs4 import BeautifulSoup

footer_address = []
footer_tel = []

for i in url['old_url']:
    ui = 'http://' + i
    f = urllib.request.urlopen(ui)
    footer_p = BeautifulSoup(f.read(), 'html.parser').find("footer").find_all('p')

    address_str = ''
    tel_str = ''
    for p in footer_p:
        if p.text[0:3] == 'โทร':
            tel_str+=p.text 
            break
        else:
            address_str+=p.text+' '

    footer_address.append(address_str)
    footer_tel.append(tel_str)

# Cleaning Data

In [89]:
new_df = url.copy()
new_df['footer_address'] = footer_address
new_df['footer_tel'] = footer_tel

In [150]:
def address(text):
    output = text.split('- ',1)[1].strip()
    return output

def tel(text):
    text = text.lower()
    output = text.split(' ')
    if len(output) > 1:
        output = output[1].strip()
    return output

def email(text):
    text = text.lower()
    output = text.split('e-mail',1)
    if len(output) > 1:
        output = output[1].strip()
        output = output.split(':')
        if len(output) > 1:
            return output[1].strip()
        else:
            return output[0].strip()
    else:
        return '-'

In [160]:
new_df['address'] = list(new_df['footer_address'].apply(address))
new_df['email'] = list(new_df['footer_tel'].apply(email))
new_df['tel'] = list(new_df['footer_tel'].apply(tel))

In [162]:
new_df.to_csv('output/address.csv', index=False)

In [163]:
new_df2 = new_df.copy()

In [165]:
del new_df2['footer_address']
del new_df2['footer_tel']

In [183]:
new_df2.to_csv('output/address_v2.csv', index=False)

In [196]:
new_df3 = pd.read_csv('output/address.csv')
new_df3['index'] = new_df3.index.values
new_df3.to_csv('output/address_v3.csv', index=False)

In [204]:
new_df3['active_url'] = 'https://' + new_df3['new_url'] + '/wp-admin/admin.php?page=acf-options'

In [205]:
# new_df3.to_csv('output/address_v3.csv', index=False)

In [217]:
new_df3 = pd.read_csv('output/address_v3.csv')
# new_df3[['email','tel','old_url'] ]